# Hypercube in N-dimension

In [57]:
import os
import sys
import scipy
import numpy as np
import pandas as pd

from colorcet import palette
import datashader as ds
import datashader.colors as dc
import datashader.utils as utils
from datashader import transfer_functions as tf

import seaborn as sns
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.animation as animation

In [3]:
import itertools as it

### Just some `matplotlib` and `seaborn` parameter tuning

In [4]:
axistitlesize = 26
axisticksize = 23
axislabelsize = 26
axislegendsize = 23
axistextsize = 20
axiscbarfontsize = 15

# Set axtick dimensions
major_size = 6
major_width = 1.2
minor_size = 3
minor_width = 1
mpl.rcParams['xtick.major.size'] = major_size
mpl.rcParams['xtick.major.width'] = major_width
mpl.rcParams['xtick.minor.size'] = minor_size
mpl.rcParams['xtick.minor.width'] = minor_width
mpl.rcParams['ytick.major.size'] = major_size
mpl.rcParams['ytick.major.width'] = major_width
mpl.rcParams['ytick.minor.size'] = minor_size
mpl.rcParams['ytick.minor.width'] = minor_width

mpl.rcParams.update({'figure.autolayout': False})

# Seaborn style settings
sns.set_style({'axes.axisbelow': True,
               'axes.edgecolor': '.8',
               'axes.facecolor': 'white',
               'axes.grid': True,
               'axes.labelcolor': '.15',
               'axes.spines.bottom': True,
               'axes.spines.left': True,
               'axes.spines.right': True,
               'axes.spines.top': True,
               'figure.facecolor': 'white',
               'font.family': ['sans-serif'],
               'font.sans-serif': ['Arial',
                'DejaVu Sans',
                'Liberation Sans',
                'Bitstream Vera Sans',
                'sans-serif'],
               'grid.color': '.8',
               'grid.linestyle': '--',
               'image.cmap': 'rocket',
               'lines.solid_capstyle': 'round',
               'patch.edgecolor': 'w',
               'patch.force_edgecolor': True,
               'text.color': '.15',
               'xtick.bottom': True,
               'xtick.color': '.15',
               'xtick.direction': 'in',
               'xtick.top': True,
               'ytick.color': '.15',
               'ytick.direction': 'in',
               'ytick.left': True,
               'ytick.right': True})

# Colorpalettes, colormaps, etc.
sns.set_palette(palette='rocket')

### Simulation hyperparameters

In [218]:
D = 4

## 0. Auxiliary functions

In [219]:
def E_mn(m, n):
    """
    Returns the hypercube element `E_mn`.
    
    Parameters:
    -----------
    m : int
        The m-dimensional sub-hypercube.
    n : int
        The n-dimensional hypercube.
    """
    return int(2**(n-m) * scipy.special.binom(n, m))

## 1. Find vertices of the N-dimensional hypercube

In [220]:
# Unit vectors in each cardinal direction
# These are only rows/columns of the D x D identity matrix
unit_vec = np.eye(D)

# Vertices of the hypercube
# These are identic to the set of all combinations of the unit vectors
## 1. Get all combinations first
tmp = []
for i in range(1, D+1):
    tmp.append(list(it.combinations(unit_vec, i)))

## 2. Calculate the vertice coordinates by summing all corresponding
##    unit vector selections
vertices = np.zeros((E_mn(m=0, n=D), D))
i = 1
for p in tmp:
    for sp in p:
        vertices[i] = np.sum(sp, axis=0)
        i += 1

## Vertices normed around zero
vertices_norm = (vertices - 0.5)#*2

In [221]:
print('Vertices of the {0}-dimensional hypercube:'.format(D))
print('=========================================')
print(vertices)

Vertices of the 4-dimensional hypercube:
[[0. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [1. 1. 0. 0.]
 [1. 0. 1. 0.]
 [1. 0. 0. 1.]
 [0. 1. 1. 0.]
 [0. 1. 0. 1.]
 [0. 0. 1. 1.]
 [1. 1. 1. 0.]
 [1. 1. 0. 1.]
 [1. 0. 1. 1.]
 [0. 1. 1. 1.]
 [1. 1. 1. 1.]]


## 2. Find edges

### 2.1. Find vertices $L=1$ step away (they're already edges basically, but doubled)

In [223]:
edges_1 = np.zeros((E_mn(m=0, n=D), D), dtype=int)

for i in range(vertices.shape[0]):
    e = 0
    for j in range(vertices.shape[0]):
        if i == j:
            continue
        if np.sum(np.abs(vertices[i]-vertices[j])) == 1:
            edges_1[i][e] = j
            e += 1

In [224]:
edges_1

array([[ 1,  2,  3,  4],
       [ 0,  5,  6,  7],
       [ 0,  5,  8,  9],
       [ 0,  6,  8, 10],
       [ 0,  7,  9, 10],
       [ 1,  2, 11, 12],
       [ 1,  3, 11, 13],
       [ 1,  4, 12, 13],
       [ 2,  3, 11, 14],
       [ 2,  4, 12, 14],
       [ 3,  4, 13, 14],
       [ 5,  6,  8, 15],
       [ 5,  7,  9, 15],
       [ 6,  7, 10, 15],
       [ 8,  9, 10, 15],
       [11, 12, 13, 14]])

### 2.2. Find (real) edges ($m=1$)

In [225]:
m = 1
edges = np.zeros((E_mn(m=m, n=D), 2**m), dtype=int)

e = 0
for i, e0 in enumerate(edges_1):
    for i2 in e0:
        edge = np.array(sorted([i, i2]))
        if (edges == edge).all(axis=1).any():
            continue
        else:
            edges[e] = edge
            e += 1

In [226]:
edges

array([[ 0,  1],
       [ 0,  2],
       [ 0,  3],
       [ 0,  4],
       [ 1,  5],
       [ 1,  6],
       [ 1,  7],
       [ 2,  5],
       [ 2,  8],
       [ 2,  9],
       [ 3,  6],
       [ 3,  8],
       [ 3, 10],
       [ 4,  7],
       [ 4,  9],
       [ 4, 10],
       [ 5, 11],
       [ 5, 12],
       [ 6, 11],
       [ 6, 13],
       [ 7, 12],
       [ 7, 13],
       [ 8, 11],
       [ 8, 14],
       [ 9, 12],
       [ 9, 14],
       [10, 13],
       [10, 14],
       [11, 15],
       [12, 15],
       [13, 15],
       [14, 15]])

## 3. Find faces

### 3.1. Find vertices $L=2$ step away

In [227]:
m = 0
edges_2 = np.zeros((E_mn(m=m, n=D), D*(D-1)//2), dtype=int)

for i in range(vertices.shape[0]):
    e = 0
    for j in range(vertices.shape[0]):
        if i == j:
            continue
        if np.sum(np.abs(vertices[i]-vertices[j])) == 2:
            edges_2[i][e] = j
            e += 1 

In [228]:
edges_2

array([[ 5,  6,  7,  8,  9, 10],
       [ 2,  3,  4, 11, 12, 13],
       [ 1,  3,  4, 11, 12, 14],
       [ 1,  2,  4, 11, 13, 14],
       [ 1,  2,  3, 12, 13, 14],
       [ 0,  6,  7,  8,  9, 15],
       [ 0,  5,  7,  8, 10, 15],
       [ 0,  5,  6,  9, 10, 15],
       [ 0,  5,  6,  9, 10, 15],
       [ 0,  5,  7,  8, 10, 15],
       [ 0,  6,  7,  8,  9, 15],
       [ 1,  2,  3, 12, 13, 14],
       [ 1,  2,  4, 11, 13, 14],
       [ 1,  3,  4, 11, 12, 14],
       [ 2,  3,  4, 11, 12, 13],
       [ 5,  6,  7,  8,  9, 10]])

### 3.2. Find intermitting vertices between vertex 0. and 2. and collect faces ($m=2$)

In [229]:
m = 2
faces = np.zeros((E_mn(m=m, n=D), 2**m), dtype=int)

f = 0
for i, e in enumerate(edges_2):
    for i2 in e:
        e0 = set(edges_1[i])
        e2 = set(edges_1[i2])
        z = np.array(sorted([i, i2] + list(e0.intersection(e2))))
        if (faces == z).all(axis=1).any():
            continue
        else:
            faces[f] = z
            f += 1

In [230]:
faces

array([[ 0,  1,  2,  5],
       [ 0,  1,  3,  6],
       [ 0,  1,  4,  7],
       [ 0,  2,  3,  8],
       [ 0,  2,  4,  9],
       [ 0,  3,  4, 10],
       [ 1,  5,  6, 11],
       [ 1,  5,  7, 12],
       [ 1,  6,  7, 13],
       [ 2,  5,  8, 11],
       [ 2,  5,  9, 12],
       [ 2,  8,  9, 14],
       [ 3,  6,  8, 11],
       [ 3,  6, 10, 13],
       [ 3,  8, 10, 14],
       [ 4,  7,  9, 12],
       [ 4,  7, 10, 13],
       [ 4,  9, 10, 14],
       [ 5, 11, 12, 15],
       [ 6, 11, 13, 15],
       [ 7, 12, 13, 15],
       [ 8, 11, 14, 15],
       [ 9, 12, 14, 15],
       [10, 13, 14, 15]])

## 4. Rotation of a 4D hypercube

In [231]:
def proj_4d(v, h):
    """
    Projects 4D coordinates to 3D from a given view distance.
    
    God bless Andrew D. Hwang:
    https://math.stackexchange.com/questions/723820/tesseract-projection-into-3d/723879#723879
    
    Parameters:
    -----------
    v : array-like
        4D array of coodinates
        
    h : distance of viewpoint from the center
    """
    
    return v[:,:3] * (h / (h - v[:,3]))[:, np.newaxis]

In [291]:
def rot_4d(v, t, p):
    """
    Rotate 4D coordinates around a hyperplane.
    """
    R_p = np.array((
        (np.cos(np.deg2rad(t)), np.sin(np.deg2rad(t))),
        (-np.sin(np.deg2rad(t)), np.cos(np.deg2rad(t)))
    ))
    I = np.eye(v.shape[1])
    
    R = I.copy()
    R[p:p+2,p:p+2] = R_p
    
    return (R @ v.T).T

In [350]:
def plot_4d(v, edges, fname):
    plt.ioff()
    fig, axes = plt.subplots(figsize=(12,12),
                         facecolor='black',
                         subplot_kw={
                             'projection' : '3d',
                             'facecolor' : 'black'
                         })
    axes.axis('off')
    axes.set_xlim(-1,1)
    axes.set_ylim(-1,1)
    axes.set_zlim(-1,1)

    axes.scatter(v[:,0],
                 v[:,1],
                 v[:,2],
                 color=cm.PuBu_r(0.85), s=15**2)

    for e in edges:
        axes.plot([ v[e[0]][0], v[e[1]][0] ],
                  [ v[e[0]][1], v[e[1]][1] ],
                  [ v[e[0]][2], v[e[1]][2] ],
                  color=cm.PuBu_r(0.5), lw=6, alpha=0.5)
        
    fig.savefig(fname,
                dpi=100,
                format='png',
                facecolor='black',
                bbox_inches='tight')
    plt.close()

In [351]:
def create_anim(v, edges, frames):

    for i in range(frames):
        t = 360/frames * i
        v_rot = rot_4d(v, t=t, p=2)
        v_proj = proj_4d(v_rot, h=1.2)
        
        index = (4-len(str(i)))*'0' + str(i)
        fname = './output/frame_{0}.png'.format(index)
        plot_4d(v=v_proj, edges=edges, fname=fname)

In [353]:
create_anim(v=vertices_norm, edges=edges, frames=300)